This script is made to calculate Welch's T-test in order to calculate confidence intervals of a comparison of simulation runs


In [4]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

simulation_folder_name = "(5,2)Comparative new" #dit moet je aanpassen
baseline_folder_name = "Baseline"
baseline = pd.read_csv("output/"+baseline_folder_name+"/performance.csv")
simulation = pd.read_csv("output/"+simulation_folder_name+"/performance.csv")

output_path = Path("comparative")
try:
    os.makedirs(output_path, exist_ok=True)
except OSError as error:
    print(error)

After the imports, we declare the code for a welch interval calculation

In [5]:
def welch_conf(x,y, add_csv = True, display_print = False, alpha = 0.0076923):
    from scipy import stats
    import math
    
    results = pd.DataFrame({'lower_bound':[], 'upper_bound':[]})
    ## Welch-Satterthwaite Degrees of Freedom ##
    dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
    lower=x.mean()-y.mean()-stats.t.ppf(1-alpha/2, dof)*math.sqrt((x.var()/x.size) +(y.var()/y.size))
    upper=x.mean()-y.mean()+stats.t.ppf(1-alpha/2, dof)*math.sqrt((x.var()/x.size) +(y.var()/y.size))
    
    if display_print:
        print("\n",
              f"lower bound= {lower:.4f}", "\n",
              f"upper bound = {upper:.4f}", "\n",
              f"Welch-Satterthwaite Degrees of Freedom= {dof:.4f}")
    if add_csv:
        results['lower_bound'] = [lower]
        results['upper_bound'] = [upper]
        results['name'] = [simulation_folder_name]
        csv_path = output_path/"results welch.csv"
        results.to_csv(csv_path, mode='a', header=False)

def paired_t(x,y, alpha =0.0076923, add_csv = True):
    from scipy import stats

    results = pd.DataFrame({'lower_bound':[], 'upper_bound':[]})
    paired_t_data = x - y #simulation - baseline
    half_width = (stats.t.ppf(1-alpha/2, paired_t_data.size)*paired_t_data.var())/np.sqrt(paired_t_data.size)
    barbar = x.mean() - y.mean()
    lower = barbar - half_width
    upper = barbar + half_width
    if add_csv:
        results['lower_bound'] = [lower]
        results['upper_bound'] = [upper]
        results['name'] = [simulation_folder_name]
        csv_path = output_path/"results paired-t.csv"
        results.to_csv(csv_path, mode='a', header=False)

Next we will run the confidence interval

In [6]:
welch_conf(baseline['objective function'], simulation['objective function'])

paired_t(baseline['objective function'], simulation['objective function'])

